# Topic Modeling

In [1]:
import pandas as pd
import numpy as np
import os
import re
import dill
import sys
from scipy.stats import uniform, randint
from string import punctuation
import nltk

# preprocessing packages
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# pipeline tools
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import FunctionTransformer

#feature selection
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

#models
from sklearn.naive_bayes import GaussianNB, MultinomialNB

#metrics
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.metrics import accuracy_score, balanced_accuracy_score

#visualization
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models

In [2]:
#nltk.download()

In [3]:
#directory locations
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
processed_data_folder = parent_directory + '/data/wine-com/processed/'
models_folder = parent_directory + '/models/'

### Load Data

In [4]:
df = pd.read_csv(processed_data_folder + '1677432096.083379.txt', 
                 sep = '|')

In [5]:
df.head()

,product_url,product_name,product_variety,product_origin,product_family,user_avg_rating,user_rating_count,winemaker_description,reviewer_name,reviewer_rating,reviewer_text
0,https://www.wine.com/product/proyecto-salvaje-...,Proyecto Salvaje del Moncayo Garnacha 2020,Grenache,"from Navarra, Spain",Red Wine,4.8,19,bright burgundy wine medium depth tobacco wild...,Decanter,92.0,part proyecto garnachas de españa collection s...
1,https://www.wine.com/product/proyecto-salvaje-...,Proyecto Salvaje del Moncayo Garnacha 2020,Grenache,"from Navarra, Spain",Red Wine,4.8,19,bright burgundy wine medium depth tobacco wild...,Wilfred Wong of Wine.com,91.0,commentary 2020 proyecto garnachas salvaje del...
2,https://www.wine.com/product/domaine-du-terme-...,Domaine du Terme Gigondas 2019,Rhone Red Blends,"from Gigondas, Rhone, France",Red Wine,4.0,17,NaN,Wine & Spirits,96.0,spectacular gigondas wine red cherry flavors s...
3,https://www.wine.com/product/domaine-du-terme-...,Domaine du Terme Gigondas 2019,Rhone Red Blends,"from Gigondas, Rhone, France",Red Wine,4.0,17,NaN,Decanter,94.0,straight first sniff clear going special soari...
4,https://www.wine.com/product/scott-harvey-moun...,Scott Harvey Mountain Selection Zinfandel 2019,Zinfandel,"from Amador, Sierra Foothills, California",Red Wine,4.3,39,fruit forward rich full flavors expressing var...,Wine Enthusiast,93.0,fresh smelling full bodied flavor packed wine ...


In [6]:
df.columns

Index(['product_url', 'product_name', 'product_variety', 'product_origin',
       'product_family', 'user_avg_rating', 'user_rating_count',
       'winemaker_description', 'reviewer_name', 'reviewer_rating',
       'reviewer_text'],
      dtype='object')

In [7]:
df.shape

(20988, 11)

### Reduce to Relevant Data

In [8]:
review_data = df[['product_family', 'reviewer_rating', 'reviewer_text']]

### Missing Data & Data Type Correction

In [9]:
review_data.isnull().sum()

product_family        0
reviewer_rating    6451
reviewer_text      6494
dtype: int64

In [10]:
review_data = review_data.dropna(subset = ['reviewer_text'])

In [11]:
review_data['reviewer_rating'] = review_data['reviewer_rating'].astype(int)

In [12]:
review_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14494 entries, 0 to 20987
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   product_family   14494 non-null  object
 1   reviewer_rating  14494 non-null  int32 
 2   reviewer_text    14494 non-null  object
dtypes: int32(1), object(2)
memory usage: 396.3+ KB


### Data Assignment & Splitting

In [13]:
# specifying predictive and target features
X = review_data.drop(columns = ['reviewer_rating'])
y = review_data[['product_family']].values.ravel()

In [14]:
# create holdout set to approximate real-world performance
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=123)

## Naive Bayes

In [15]:
class CovertToList(TransformerMixin):
    def transform(self, X):
        transformed_data = []
        #transform to dataframe
        X = pd.DataFrame(X)
        #get colnames
        colnames = X.columns
        #iterate through columns
        for col in colnames:
            X = X[col].tolist()
            X = [str(i) for i in X]
            transformed_data.extend(X)
        return np.array(transformed_data)

    def fit(self, X, y=None):
        return self

In [16]:
# specifiying column transformer fields
text_variables = ['reviewer_text']

# Count Vectorizer pipeline:
cv_transformer = Pipeline([('convert_to_list', CovertToList()),
                           ('count_vectorizer', CountVectorizer())])

nb_full_pipeline = Pipeline([('column_transformer', ColumnTransformer([('text', cv_transformer, text_variables)],
                                                                      remainder = 'drop')),
                             ('near_zero_variance', VarianceThreshold()),
                             ('naive_bayes', MultinomialNB())])

In [17]:
search_space = [{'naive_bayes__alpha': uniform(0.001, 10.0),
                 'column_transformer__text__count_vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)]}]


cv_nb = RandomizedSearchCV(nb_full_pipeline,
                            param_distributions = search_space, 
                            n_iter = 10, 
                            cv = 5,
                            n_jobs = 6,
                            scoring = 'accuracy',
                            random_state=123)

cv_nb.fit(X_train, y_train)

print("Best parameter (CV score=%0.3f):" % cv_nb.best_score_)
print(cv_nb.best_params_)

Best parameter (CV score=0.897):
{'column_transformer__text__count_vectorizer__ngram_range': (2, 2), 'naive_bayes__alpha': 0.5977789660956835}


### Write Pipeline to File

In [18]:
cv_nb_best_pipeline = cv_nb.best_estimator_
with open(models_folder + 'topic_nb_best_cv.pkl', 'wb') as f:
    dill.dump(cv_nb_best_pipeline, f)

### Holdout Performance

In [19]:
accuracy_score(y_test, cv_nb.predict(X_test))

0.9061745429458434

## LDA - TF-IDF Vectorizer

In [20]:
# TF-IDF  pipeline:
tf_transformer = Pipeline([('convert_to_list', CovertToList()),
                           ('tfidf_vectorizer', TfidfVectorizer())])

tf_full_pipeline = Pipeline([('column_transformer', ColumnTransformer([('text', tf_transformer, text_variables)],
                                                                      remainder = 'drop')),
                             ('near_zero_variance', VarianceThreshold()),
                             ('naive_bayes', MultinomialNB())])

In [21]:
search_space = [{'naive_bayes__alpha': uniform(0.001, 10.0),
                 'column_transformer__text__tfidf_vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)]}]

tf_nb = RandomizedSearchCV(tf_full_pipeline,
                            param_distributions = search_space, 
                            n_iter = 10, 
                            cv = 5,
                            n_jobs = 6,
                            scoring = 'accuracy',
                            random_state=123)
tf_nb.fit(X_train, y_train)

print("Best parameter (CV score=%0.3f):" % tf_nb.best_score_)
print(tf_nb.best_params_)

Best parameter (CV score=0.728):
{'column_transformer__text__tfidf_vectorizer__ngram_range': (2, 2), 'naive_bayes__alpha': 0.5977789660956835}


In [22]:
tf_nb_best_pipeline = tf_nb.best_estimator_
with open(models_folder + 'topic_nb_best_tfidf.pkl', 'wb') as f:
    dill.dump(tf_nb_best_pipeline, f)

### Holdout Performance

In [23]:
accuracy_score(y_test, tf_nb.predict(X_test))

0.7419799931010693